# Data Extraction Documentation
## BS Data Science Project - Recurrent Bitcoin Network

This notebook contains a brief documentation on retrieving data from [Messari](https://messari.io/) through its [API](https://messari.io/api/docs). Method of data collection would be through a REST API with a `json` response.

### Data

The expected data would be a data frame of time series data denoted by its columns and it would be splitted, normalized, and be fed for model training and testing. In this project, we would only use daily time series data from 2016 to 2020 as the training and validation set, while 2021 would be used as a test set. Note that this notebook does not include the retrieval of the test set.

### Prerequesites

Before running this notebook, several python packages are needed to be installed as shown in the following cell. In addition, changing of working directory within the local repository was done.

In [1]:
%%capture

# Redirects the current working directory to `/src/` directory
%cd ..\src

In [2]:
# Built-in packages
import requests
import json
import time
from datetime import datetime

# Packages to be installed
import pandas as pd
from tqdm import tqdm

# Local python functions
from messari import load

### Retrieve Metrics Data From Messari

This section retrieves the complete list of the possible metrics to be queried in time series.

In [3]:
# Creates a session to save cookies and headers
sess = requests.session()

In [4]:
%%time
metrics = sess.get('https://data.messari.io/api/v1/assets/metrics')
metrics

Wall time: 3.24 s


<Response [200]>

In [5]:
df_metrics = pd.DataFrame(metrics.json()['data']['metrics'])
df_metrics.head()

,metric_id,name,description,values_schema,minimum_interval,role_restriction,source_attribution
0,exch.binance.flow.net.ntv,Binance Net Flows (Native Units),The net native units value sent or withdrawn t...,{'exchange_flow_net_binance': 'The net native ...,1d,"[pro, enterprise]","[{'name': 'Coinmetrics', 'url': 'https://coinm..."
1,exch.poloniex.flow.net.ntv,Poloniex Net Flows (Native Units),The net native units value sent or withdrawn t...,{'exchange_flow_net_poloniex': 'The net native...,1d,"[pro, enterprise]","[{'name': 'Coinmetrics', 'url': 'https://coinm..."
2,exch.sply.binance,Supply on Binance,The sum USD value held by Binance at the end o...,{'exchange_supply_binance_usd': 'The sum USD v...,1d,"[pro, enterprise]","[{'name': 'Coinmetrics', 'url': 'https://coinm..."
3,addr.bal.100.ntv.cnt,Addresses with balance greater than 100 native...,The sum count of unique addresses holding at l...,{'addresses': 'The sum count of unique address...,1d,"[pro, enterprise]","[{'name': 'Coinmetrics', 'url': 'https://coinm..."
4,exch.binance.flow.net,Binance Net Flows,The net USD value sent or withdrawn to/from Bi...,{'exchange_flow_net_binance_usd': 'The net USD...,1d,"[pro, enterprise]","[{'name': 'Coinmetrics', 'url': 'https://coinm..."


Observing on the `role_restriction` column, there are missing values given. However, these missing values indicate that the given metrics has no restriction. Therefore, we would select all metrics with missing values on `role_restriction` and save it under a `csv` file.

In [6]:
free_metrics = df_metrics[df_metrics['role_restriction'].isna()]
free_metrics.reset_index(drop=True, inplace=True)

In [7]:
free_metrics.head()

,metric_id,name,description,values_schema,minimum_interval,role_restriction,source_attribution
0,exch.flow.in.ntv.incl,Deposits on Exchanges - Inclusive (Native Units),The amount of the asset sent to exchanges that...,{'flow_in': 'The amount of the asset sent to e...,1d,NaN,"[{'name': 'Coinmetrics', 'url': 'https://coinm..."
1,exch.flow.in.ntv,Deposits on Exchanges (Native Units),The amount of the asset sent to exchanges that...,{'flow_in': 'The amount of the asset sent to e...,1d,NaN,"[{'name': 'Coinmetrics', 'url': 'https://coinm..."
2,sply.total.iss,Total Issuance,The sum USD value of all new native units issu...,{'issuance_total_usd': 'The sum USD value of a...,1d,NaN,"[{'name': 'Coinmetrics', 'url': 'https://coinm..."
3,reddit.active.users,Reddit Active Users,The number of active users on the asset's prim...,{'active_users': ' The number of active users ...,1h,NaN,"[{'name': 'reddit', 'url': 'https://reddit.com..."
4,exch.sply.usd,Supply on Exchanges,The sum USD value of all native units held in ...,{'supply_usd': 'The sum USD value of all nativ...,1d,NaN,"[{'name': 'Coinmetrics', 'url': 'https://coinm..."


In [8]:
free_metrics.to_csv('../raw/messari_freemetrics.csv', index=False)

In [9]:
with open('_metrics.json', 'w') as f:
    json.dump(
        free_metrics[['metric_id','name']].to_json(orient='records'),
        f,
        indent=4
    )

### Retrieve Bitcoin Time Series

In this section, we would retrieve time series data with the given metrics retrieved above. The `tqdm` progress bar package was implemented to show the data collection process.

In [10]:
data = load('../raw/rawdata.csv')

Now retrieving: cg.sply.circ:  26%|██▋       | 15/57 [01:08<02:22,  3.40s/it]

Request [500] in cg.sply.circ


Now retrieving: sply.liquid:  82%|████████▏ | 47/57 [04:08<00:29,  2.93s/it]

Request [501] in sply.liquid


Now retrieving: txn.tfr.val.med.ntv: 100%|██████████| 57/57 [04:52<00:00,  5.13s/it]


In [11]:
data.head()

,exch.flow.in.ntv.incl - flow_in,exch.flow.in.ntv - flow_in,sply.total.iss - issuance_total_usd,reddit.active.users - active_users,exch.sply.usd - supply_usd,exch.flow.out.usd.incl - flow_out_usd,txn.fee.avg - transaction_fee_avg,txn.tsfr.cnt - transfers_count,txn.cnt - transaction_count,sply.circ - circulating_supply,...,daily.vol - volatility_90d,daily.vol - volatility_1yr,daily.vol - volatility_3yr,mcap.dom - marketcap_dominance,txn.tfr.val.adj.ntv - transaction_volume_adjusted,blk.cnt - block_count,nvt.adj.90d.ma - nvt_adjusted_90d_ma,sply.out - supply_outstanding,reddit.subscribers - subscribers,txn.tfr.val.med.ntv - transfer_value_median
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-01-01,32181.171652,30112.798700,1.467038e+06,NaN,2.090564e+08,1.397067e+07,0.069424,320580,124427,1.503561e+07,...,0.677785,0.685757,1.028978,94.142223,174515.818449,135,41.147256,1.503289e+07,NaN,0.011701
2016-01-02,26274.545676,24549.029482,1.574845e+06,NaN,2.091767e+08,1.117990e+07,0.091773,405602,147864,1.503924e+07,...,0.678005,0.685251,1.028922,94.131509,189629.972730,145,41.048731,1.503651e+07,NaN,0.006900
2016-01-03,39479.215335,36097.173063,1.559244e+06,NaN,2.103291e+08,1.375688e+07,0.072584,432531,145029,1.504286e+07,...,0.678520,0.685339,1.028857,94.116662,232482.193500,145,40.644184,1.504014e+07,NaN,0.005126
2016-01-04,45876.869599,41544.044513,1.961357e+06,NaN,2.111916e+08,2.064268e+07,0.073746,419538,178576,1.504739e+07,...,0.678528,0.675956,1.028857,94.080515,258630.297507,181,40.918827,1.504466e+07,NaN,0.017405
2016-01-05,51377.988662,48262.028042,1.698228e+06,NaN,2.038569e+08,2.918461e+07,0.076591,438135,183659,1.505132e+07,...,0.677822,0.673562,1.028863,94.131122,335747.819336,157,40.754906,1.504859e+07,NaN,0.019620


In [12]:
topna = data.isna().sum().sort_values(ascending=False)[:10]
topna

txn.tfr.erc20.cnt - transaction_transfer_count_erc20      1827
txn.tfr.erc721.cnt - transaction_transfer_count_erc721    1827
reddit.subscribers - subscribers                          1588
reddit.active.users - active_users                        1588
nvt.adj - nvt_adjusted                                       0
txn.fee.med.ntv - transaction_fee_median                     0
hashrate - hash_rate                                         0
txn.vol - transaction_volume_usd                             0
exch.flow.in.usd.incl - flow_in_usd                          0
txn.tfr.avg.ntv - transfer_value_avg                         0
dtype: int64

Missing data in this project would be dropped as the model requires complete data from 2016 to 2020.

In [13]:
data2 = data.drop(topna.index[[bool(a) for a in topna.values.tolist()]], axis=1)
data2.head()

,exch.flow.in.ntv.incl - flow_in,exch.flow.in.ntv - flow_in,sply.total.iss - issuance_total_usd,exch.sply.usd - supply_usd,exch.flow.out.usd.incl - flow_out_usd,txn.fee.avg - transaction_fee_avg,txn.tsfr.cnt - transfers_count,txn.cnt - transaction_count,sply.circ - circulating_supply,daily.shp - sharpe_30d,...,daily.vol - volatility_30d,daily.vol - volatility_90d,daily.vol - volatility_1yr,daily.vol - volatility_3yr,mcap.dom - marketcap_dominance,txn.tfr.val.adj.ntv - transaction_volume_adjusted,blk.cnt - block_count,nvt.adj.90d.ma - nvt_adjusted_90d_ma,sply.out - supply_outstanding,txn.tfr.val.med.ntv - transfer_value_median
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-01-01,32181.171652,30112.798700,1.467038e+06,2.090564e+08,1.397067e+07,0.069424,320580,124427,1.503561e+07,3.254284,...,0.640308,0.677785,0.685757,1.028978,94.142223,174515.818449,135,41.147256,1.503289e+07,0.011701
2016-01-02,26274.545676,24549.029482,1.574845e+06,2.091767e+08,1.117990e+07,0.091773,405602,147864,1.503924e+07,3.742549,...,0.631588,0.678005,0.685251,1.028922,94.131509,189629.972730,145,41.048731,1.503651e+07,0.006900
2016-01-03,39479.215335,36097.173063,1.559244e+06,2.103291e+08,1.375688e+07,0.072584,432531,145029,1.504286e+07,3.463730,...,0.633846,0.678520,0.685339,1.028857,94.116662,232482.193500,145,40.644184,1.504014e+07,0.005126
2016-01-04,45876.869599,41544.044513,1.961357e+06,2.111916e+08,2.064268e+07,0.073746,419538,178576,1.504739e+07,3.520499,...,0.633926,0.678528,0.675956,1.028857,94.080515,258630.297507,181,40.918827,1.504466e+07,0.017405
2016-01-05,51377.988662,48262.028042,1.698228e+06,2.038569e+08,2.918461e+07,0.076591,438135,183659,1.505132e+07,2.312140,...,0.592641,0.677822,0.673562,1.028863,94.131122,335747.819336,157,40.754906,1.504859e+07,0.019620


In [14]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1827 entries, 2016-01-01 to 2020-12-31
Freq: D
Data columns (total 61 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   exch.flow.in.ntv.incl - flow_in                    1827 non-null   float64
 1   exch.flow.in.ntv - flow_in                         1827 non-null   float64
 2   sply.total.iss - issuance_total_usd                1827 non-null   float64
 3   exch.sply.usd - supply_usd                         1827 non-null   float64
 4   exch.flow.out.usd.incl - flow_out_usd              1827 non-null   float64
 5   txn.fee.avg - transaction_fee_avg                  1827 non-null   float64
 6   txn.tsfr.cnt - transfers_count                     1827 non-null   int64  
 7   txn.cnt - transaction_count                        1827 non-null   int64  
 8   sply.circ - circulating_supply                     1827 non-nu

In [15]:
data2.to_csv('../raw/cleandata.csv')